In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import json
import skimage
import imgaug
import imgaug.augmenters as iaa

from tqdm import tqdm


# Root directory of the project
ROOT_DIR = os.path.abspath("./")

from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

from moles import MolesConfig
from moles import MolesDataset
from moles import BalancedDataset
from moles import BalancedDatasetFast
from moles import ISIC17Dataset
from moles import ISIC17AugDataset

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

Using TensorFlow backend.


In [2]:
config = MolesConfig()

In [3]:
dataset_train = ISIC17AugDataset()
dataset_train.load_moles("data/train/")
dataset_train.prepare()
ds = dataset_train

dataset_val = ISIC17Dataset()
dataset_val.load_moles("../Data/val")
dataset_val.prepare()

In [ ]:
len(ds.image_ids)

In [ ]:
ds = dataset_train

# Load and display random samples
image_ids = np.random.choice(ds.image_ids, 4)
for image_id in image_ids:
    image = ds.load_image(image_id)
    mask, class_ids = ds.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, ds.class_names)

In [4]:
augmentation = iaa.SomeOf((5,8), [
    iaa.Fliplr(1.0),
    iaa.OneOf([
        iaa.GaussianBlur((1, 6.0)),
        iaa.AverageBlur(k=(1, 3)),
        iaa.MedianBlur(k=(3, 5)),
    ]),
    iaa.Flipud(1.0),
    iaa.Rot90((1,3)),
    iaa.CropAndPad(percent=(-0.25, 0.25)),
    iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.2),
    iaa.OneOf([iaa.Dropout((0.01, 0.1), per_channel=0.5),
               iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),]),
    iaa.PiecewiseAffine(scale=(0.01, 0.05)),
])

In [5]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead


In [6]:
COCO_MODEL_PATH = ROOT_DIR + "/mrcnn/mask_rcnn_coco.h5"
LATEST_MODEL_PATH = ROOT_DIR + "/logs/moles20200724T1454/mask_rcnn_moles_0165.h5"

model.load_weights(LATEST_MODEL_PATH, by_name=True,
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                            "mrcnn_bbox", "mrcnn_mask"])
model.keras_model.metrics_tensors = []

Re-starting from epoch 165


In [7]:
# Train top layers
model.train(ds, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=70, 
            layers='heads',
            augmentation=augmentation
            )


Starting at epoch 60. LR=0.001

Checkpoint Path: /home/useydaninja/projects/melanoma/Skin-Cancer-Segmentation/logs/moles20200724T1454/mask_rcnn_moles_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_co

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "







Epoch 61/70
400/400 [==============================] - 1086s 3s/step - loss: 1.2096 - val_loss: 0.5391

Epoch 62/70
400/400 [==============================] - 789s 2s/step - loss: 0.8976 - val_loss: 0.8194
Epoch 63/70
400/400 [==============================] - 964s 2s/step - loss: 0.8545 - val_loss: 0.6010
Epoch 64/70
400/400 [==============================] - 953s 2s/step - loss: 0.8815 - val_loss: 0.5619
Epoch 65/70
400/400 [==============================] - 964s 2s/step - loss: 0.8828 - val_loss: 0.7418
Epoch 66/70
400/400 [==============================] - 966s 2s/step - loss: 0.8714 - val_loss: 0.5810
Epoch 67/70
400/400 [==============================] - 963s 2s/step - loss: 0.8761 - val_loss: 0.5334
Epoch 68/70
400/400 [==============================] - 973s 2s/step - loss: 0.8569 - val_loss: 0.5017
Epoch 69/70
400/400 [==============================] - 962s 2s/step - loss: 0.8545 - val_loss: 0.6930
Epoch 70/70
400/400 [==============================] - 969s 2s/step - loss:

In [ ]:
model.train(ds, dataset_val,
            learning_rate=config.LEARNING_RATE/3,
            epochs=60,
            augmentation=augmentation,
            layers="4+")


Starting at epoch 45. LR=0.0003333333333333333

Checkpoint Path: /home/useydaninja/projects/melanoma/Skin-Cancer-Segmentation/logs/moles20200724T1454/mask_rcnn_moles_{epoch:04d}.h5
Selecting layers to train
res4a_branch2a         (Conv2D)
bn4a_branch2a          (BatchNorm)
res4a_branch2b         (Conv2D)
bn4a_branch2b          (BatchNorm)
res4a_branch2c         (Conv2D)
res4a_branch1          (Conv2D)
bn4a_branch2c          (BatchNorm)
bn4a_branch1           (BatchNorm)
res4b_branch2a         (Conv2D)
bn4b_branch2a          (BatchNorm)
res4b_branch2b         (Conv2D)
bn4b_branch2b          (BatchNorm)
res4b_branch2c         (Conv2D)
bn4b_branch2c          (BatchNorm)
res4c_branch2a         (Conv2D)
bn4c_branch2a          (BatchNorm)
res4c_branch2b         (Conv2D)
bn4c_branch2b          (BatchNorm)
res4c_branch2c         (Conv2D)
bn4c_branch2c          (BatchNorm)
res4d_branch2a         (Conv2D)
bn4d_branch2a          (BatchNorm)
res4d_branch2b         (Conv2D)
bn4d_branch2b          

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "







Epoch 46/60
400/400 [==============================] - 1123s 3s/step - loss: 1.1044 - val_loss: 0.6485

Epoch 47/60
400/400 [==============================] - 790s 2s/step - loss: 0.9605 - val_loss: 0.4783
Epoch 48/60
400/400 [==============================] - 989s 2s/step - loss: 0.9139 - val_loss: 0.4280
Epoch 49/60
400/400 [==============================] - 990s 2s/step - loss: 0.9300 - val_loss: 0.8523
Epoch 50/60
400/400 [==============================] - 989s 2s/step - loss: 0.8987 - val_loss: 0.6503
Epoch 51/60
400/400 [==============================] - 994s 2s/step - loss: 0.9019 - val_loss: 0.6378
Epoch 52/60
400/400 [==============================] - 997s 2s/step - loss: 0.9057 - val_loss: 0.7882
Epoch 53/60
400/400 [==============================] - 984s 2s/step - loss: 0.8871 - val_loss: 0.8204
Epoch 54/60
400/400 [==============================] - 991s 2s/step - loss: 0.8725 - val_loss: 0.6579
Epoch 55/60
215/400 [===============>..............] - ETA: 7:36 - loss: 0.

In [ ]:
model.train(ds, dataset_val,
            learning_rate=config.LEARNING_RATE/10,
            epochs=100,
            augmentation=augmentation,
            layers="all")


Starting at epoch 80. LR=0.0001

Checkpoint Path: /home/useydaninja/projects/melanoma/Skin-Cancer-Segmentation/logs/moles20200724T1454/mask_rcnn_moles_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "







Epoch 81/100
400/400 [==============================] - 1216s 3s/step - loss: 1.9166 - val_loss: 0.8639

Epoch 82/100
400/400 [==============================] - 901s 2s/step - loss: 0.8757 - val_loss: 0.4724
Epoch 83/100
400/400 [==============================] - 1111s 3s/step - loss: 0.7587 - val_loss: 0.3551
Epoch 84/100
400/400 [==============================] - 1110s 3s/step - loss: 0.7388 - val_loss: 0.5598
Epoch 85/100
400/400 [==============================] - 1112s 3s/step - loss: 0.7448 - val_loss: 0.4266
Epoch 86/100
400/400 [==============================] - 1102s 3s/step - loss: 0.7165 - val_loss: 0.4547
Epoch 87/100
400/400 [==============================] - 1103s 3s/step - loss: 0.7183 - val_loss: 0.5872
Epoch 88/100
400/400 [==============================] - 1102s 3s/step - loss: 0.7088 - val_loss: 0.5445
Epoch 89/100
400/400 [==============================] - 1113s 3s/step - loss: 0.6982 - val_loss: 0.4994
Epoch 90/100
400/400 [==============================] - 110

In [ ]:
model.train(ds, dataset_val,
            learning_rate=config.LEARNING_RATE/10,
            epochs=190,
            augmentation=augmentation,
            layers="all")


Starting at epoch 165. LR=0.0001

Checkpoint Path: /home/useydaninja/projects/melanoma/Skin-Cancer-Segmentation/logs/moles20200724T1454/mask_rcnn_moles_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
re

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
